In [111]:
import sympy,time
from sympy.matrices import Matrix #最后处理矩阵用
#导入工具包
load('pkg/Lie.sage') #李代数相关的工具
load('pkg/spo.sage') #李超代数spo的工具
load('pkg/MatSp.sage') #处理矩阵的工具
load('pkg/AlgebraBSC.sage') #结构常数相关工具
load('pkg/pbw.sage') #李代数pbw基的工具
load('pkg/rex.py') #解方程等相关工具
class PBWElement(PBWElement):
    def linear_rep(self,elements):
        '''将elements用pbw基+矩阵线性表示，elements为元素或符号变量元组'''
        if isinstance(elements,tuple): #若输入元组，进行转格式
            elements = [self.tuple2element(element) for element in elements ]
        keys = set()
        for ele in elements:
            keys = keys.union(ele.keys)
        mat = matrix([[ele.element[key] if key in ele else 0 for ele in elements] for key in keys])
        basis = [self({key:1}) for key in keys]
        return mat,basis
load('pkg/pbws.sage') #李超代数pbw基的工具

In [112]:
#V初始化数据
tt = time.time()
t,k = 1,2
L = SpO(t,k) #李超代数
mats = L.negative_matrixs + L.h_matrixs + L.positive_matrixs #矩阵信息
m = len(L.positive_roots)
print('李超代数：spo(%d,%d)\n空间维数为：%d\n正根数目为：%d'%(2*t,2*k,len(mats),m))
es = Lie.symbols('e',m)
fs = Lie.symbols('f',m)
hs = Lie.symbols('h',k+t) 
syms = fs + hs +es #用作结构常数的符号变量
#奇根对应的符号变量
roots = L.positive_roots 
ind1 = [roots.index(r) for r in L.odd_roots if r in roots]
roots = L.simple_roots
ind2 = [roots.index(r) for r in L.odd_roots if r in roots]
odds = [fs[i] for i in ind1] + [hs[i] for i in ind2] + [es[i] for i in ind1] 

#计算结构常数
print('计算结构常数')
N = AlgebraBSC.lie_sup_SC(mats,L.odd_mats)
'''
Lf = {i+1:Element(element,N,syms) for i,element in enumerate(fs)}
Le = {i+1:Element(element,N,syms) for i,element in enumerate(es)}
Lh = {i+1:Element(element,N,syms) for i,element in enumerate(hs)}
#'''
#pbw基
print('计算李超代数泛包络的pbw基')
vf = {i+1:SPBWElement(element,N,syms,odds) for i,element in enumerate(fs)}
ve = {i+1:SPBWElement(element,N,syms,odds) for i,element in enumerate(es)}
vh = {i+1:SPBWElement(element,N,syms,odds) for i,element in enumerate(hs)}
ide = SPBWElement({tuple():1},N,syms,odds) #幺元
vectors = L.vectors
print('初始化完毕，用时 %.3fs'%(time.time()-tt))

#权的正根分解
tt = time.time()
weight = L.basis[0] + L.basis[1];print('待讨论的权：',weight) #待讨论的权（正交基形式）
root = L.vector2root(weight) #转单根形式
dec = L.positive_dec(root) #正根分解（元组构成）
negative_roots = L.negative_roots #负根
dec_root = []
for line in dec:
    new = []
    for i,element in enumerate(line):
        if element:
            new = new + [negative_roots[i]]*element
    dec_root.append(tuple(new))
dec_vector = [tuple(L.root2vector(r) for r in line) for line in dec_root]
print('权空间维数：',len(dec)) #分解总数目
print('用时：%.3fs'%(time.time()-tt))
#for line in res_vectors:print(line)
table([[i for i in line if i] for line in dec_vector]) #分解结果

李超代数：spo(2,4)
空间维数为：17
正根数目为：7
计算结构常数
计算李超代数泛包络的pbw基
初始化完毕，用时 0.534s
待讨论的权： d + e1
权空间维数： 4
用时：0.018s


-d + e1
  -e1 + e2
  -e1 - e2
  -d - e1

In [113]:
#相应的pbw基
basis = [tuple(fs[negative_roots.index(i)] for i in line) for line in dec_root]
pbw_basis = [ide.tuple2element(i) for i in basis]
print('权空间对应的pbw基：')
pbw_basis

权空间对应的pbw基：


[{(f1, f2, f3): 1}, {(f2, f5): 1}, {(f3, f4): 1}, {(f6,): 1}]

In [114]:
#最高权向量
x = Lie.symbols('x',k)
la = (x[0]+2*k-2)*L.basis[0] + sum([i*j for i,j in zip(x,L.basis[1:])])
print('最高权：',la)

#泛包络上计算结果res
tt = time.time()
res = [[ve[i]*element for element in pbw_basis] for i in range(1,k+t+1)]
print("计算泛包络上的乘积结果，乘积总数：",(k+t)*len(pbw_basis))
print('用时：%.3fs\n'%(time.time()-tt))

#化简为Verma模的元素
tt = time.time()
res_verma = [[i.to_verma(la,L,k+t) for i in line] for line in res]
print('转化为Verma模上结果，用时：%.3fs\n'%(time.time()-tt))


最高权： d*(x1 + 2) + e1*x1 + e2*x2
计算泛包络上的乘积结果，乘积总数： 12
用时：0.967s

转化为Verma模上结果，用时：0.211s



In [117]:

#转化为矩阵
tt = time.time()
data = []
for line in res_verma:
    mat,_ = ide.linear_rep(line)
    data.append(mat)
mat = matrix.block(k+t,1,data)
mat_sym = Matrix(mat)
if k<=3:
    print('矩阵的秩为',mat_sym.rank()) #54打印时间太长了！
    if k==2:
        solution = solve_matrix(mat_sym) #sage上的运算时间太长了！
print('矩阵阶数为 %d*%d  '%mat.dimensions(),end='')
print('计算用时：%.3fs'%(time.time()-tt))
#save_vari(mat_sym,'mat%d.pydata'%k) #保存数据

矩阵的秩为 3


NameError: name 'solve_matrix' is not defined

In [116]:
ele

{(f1, f2): 1}

In [ ]:
'''计算时间
2阶：1秒左右 5*4
3阶：14秒左右 23*15
4阶：134秒左右 106*55
5阶：1116秒左右 462*200
'''

In [ ]:
#matlab矩阵命令
print(symmat_to_matlab(mat_sym,k))

In [ ]:
#matlab解方程组的命令
print(symmat_to_command(mat_sym,k))

In [ ]:
#matlab解信息的命令
print(read_solution(len(dec)-1))

In [19]:
a.

{(f1,): 1}